# QTAlgo Super26 Strategy Analysis

This notebook demonstrates how to use the walk-forward optimization framework.

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import yaml
from pathlib import Path

from src.strategy.indicators import calculate_all_indicators
from src.strategy.signals import generate_signals
from src.strategy.exits import simulate_exits
from src.data.loader import DataLoader
from src.optimization.walk_forward import WalkForwardOptimizer
from src.optimization.parameter_space import ParameterSpace, load_base_parameters, merge_parameters
from src.optimization.metrics import calculate_all_metrics
from src.utils.plotting import (
    plot_strategy_performance, plot_walk_forward_efficiency,
    plot_trade_analysis, plot_signal_distribution
)
from src.utils.reporting import generate_performance_report

import warnings
warnings.filterwarnings('ignore')

## 1. Load Configuration and Data

In [ ]:
# Load strategy parameters
strategy_config_path = '../config/strategy_params.yaml'
with open(strategy_config_path, 'r') as f:
    strategy_config = yaml.safe_load(f)

# Load base parameters
base_params = load_base_parameters(strategy_config_path)

print("Strategy Parameters Loaded")
print(f"Number of parameters: {len(base_params)}")

In [ ]:
# Load sample data (replace with your data path)
loader = DataLoader()

# Option 1: Load from CSV
# df = loader.load_csv('../data/raw/BTCUSD.csv', 'BTCUSD')

# Option 2: Generate sample data for demo
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='1H')
np.random.seed(42)
close = 10000 + np.cumsum(np.random.randn(len(dates)) * 50)
high = close + np.random.rand(len(dates)) * 100
low = close - np.random.rand(len(dates)) * 100
open_price = close + np.random.randn(len(dates)) * 20
volume = np.random.randint(1000, 100000, len(dates))

df = pd.DataFrame({
    'open': open_price,
    'high': high,
    'low': low,
    'close': close,
    'volume': volume
}, index=dates)

df = loader.clean_data(df)

print(f"Data loaded: {len(df)} rows")
print(f"Date range: {df.index[0]} to {df.index[-1]}")
df.head()

## 2. Single Backtest with Default Parameters

In [ ]:
# Calculate indicators
df_with_indicators = calculate_all_indicators(df, base_params)

print("Indicators calculated")
print(f"Columns: {list(df_with_indicators.columns)}")

In [ ]:
# Generate signals
df_with_signals = generate_signals(df_with_indicators, base_params)

print(f"Signals generated")
print(f"Long signals: {(df_with_signals['signal'] == 1).sum()}")
print(f"Short signals: {(df_with_signals['signal'] == -1).sum()}")
print(f"No signal: {(df_with_signals['signal'] == 0).sum()}")

In [ ]:
# Simulate trades
trades_df = simulate_exits(df, base_params, df_with_signals)

print(f"Total trades: {len(trades_df)}")
trades_df.head(10)

In [ ]:
# Calculate performance metrics
metrics = calculate_all_metrics(trades_df)

# Generate report
report = generate_performance_report(metrics, trades_df)
print(report)

## 3. Visualizations

In [ ]:
# Calculate equity curve
from src.optimization.metrics import calculate_returns
equity = calculate_returns(trades_df, initial_capital=10000)

# Plot strategy performance
fig = plot_strategy_performance(df, trades_df, equity)
fig.show()

In [ ]:
# Plot trade analysis
fig = plot_trade_analysis(trades_df)
fig.show()

In [ ]:
# Plot signal distribution
fig = plot_signal_distribution(df_with_signals)
fig.show()

## 4. Walk-Forward Optimization

In [ ]:
# Setup parameter space
param_space = ParameterSpace(strategy_config_path)

print(f"Optimizing {len(param_space.parameters)} parameters")
print(param_space.get_parameter_info())

In [ ]:
# Create strategy function
def strategy_function(data, params):
    merged_params = merge_parameters(base_params, params)
    df_ind = calculate_all_indicators(data, merged_params)
    df_sig = generate_signals(df_ind, merged_params)
    trades = simulate_exits(data, merged_params, df_sig)
    return trades

In [ ]:
# Setup walk-forward optimizer
wf_config = {
    'mode': 'rolling',
    'train_period_months': 12,
    'test_period_months': 3,
    'step_months': 3,
    'algorithm': 'optuna',
    'n_trials': 50,  # Reduced for faster execution
    'n_jobs': -1,
    'objectives': {
        'sharpe_ratio': {'weight': 0.4, 'direction': 'maximize'},
        'max_drawdown': {'weight': 0.3, 'direction': 'minimize'},
        'win_rate': {'weight': 0.2, 'direction': 'maximize'},
        'profit_factor': {'weight': 0.1, 'direction': 'maximize'}
    }
}

optimizer = WalkForwardOptimizer(wf_config)

In [ ]:
# Run walk-forward optimization (this may take a while)
print("Starting walk-forward optimization...")
periods = optimizer.run_walk_forward(df, param_space, strategy_function)
print(f"Optimization complete. Processed {len(periods)} periods.")

In [ ]:
# Calculate walk-forward efficiency
wfe = optimizer.calculate_wfe()
print(f"Walk-Forward Efficiency: {wfe:.2f}")

# Get aggregate results
results = optimizer.get_aggregate_results()
print("\nAggregate Results:")
print(results)

In [ ]:
# Analyze parameter stability
param_stability = optimizer.analyze_parameter_stability()
print("Parameter Stability:")
print(param_stability)

In [ ]:
# Plot walk-forward results
fig = plot_walk_forward_efficiency(periods)
fig.show()

In [ ]:
# Generate walk-forward report
from src.utils.reporting import generate_walk_forward_report

wf_report = generate_walk_forward_report(periods, wfe, param_stability)
print(wf_report)

## 5. Export Results

In [ ]:
# Save results
output_dir = Path('../results')
output_dir.mkdir(exist_ok=True)

# Save optimizer state
optimizer.save_results(output_dir / 'wf_results.pkl')

# Save parameter stability
if param_stability is not None:
    param_stability.to_csv(output_dir / 'param_stability.csv')

print("Results saved to results/ directory")